In [2]:
import os
import cv2
import dlib
import math
import numpy as np
from skimage.feature import hog
from PIL import Image
from matplotlib import pyplot as plt

# 逆时针旋转
def Nrotate(angle, valuex, valuey, pointx, pointy):
    angle = (angle/180)*math.pi
    valuex = np.array(valuex)
    valuey = np.array(valuey)
    nRotatex = (valuex-pointx)*math.cos(angle) - \
        (valuey-pointy)*math.sin(angle) + pointx
    nRotatey = (valuex-pointx)*math.sin(angle) + \
        (valuey-pointy)*math.cos(angle) + pointy
    return (nRotatex, nRotatey)
# 顺时针旋转
def Srotate(angle, valuex, valuey, pointx, pointy):
    angle = (angle/180)*math.pi
    valuex = np.array(valuex)
    valuey = np.array(valuey)
    sRotatex = (valuex-pointx)*math.cos(angle) + \
        (valuey-pointy)*math.sin(angle) + pointx
    sRotatey = (valuey-pointy)*math.cos(angle) - \
        (valuex-pointx)*math.sin(angle) + pointy
    return (sRotatex, sRotatey)
# 将四个点做映射
def rotatecordiate(angle, rectboxs, pointx, pointy):
    output = []
    for rectbox in rectboxs:
        if angle > 0:
            output.append(
                Srotate(angle, rectbox[0], rectbox[1], pointx, pointy))
        else:
            output.append(
                Nrotate(-angle, rectbox[0], rectbox[1], pointx, pointy))
    return output
# 裁剪图片
def imagecrop(image, box,i,j):
    xs = [x[1] for x in box]
    ys = [x[0] for x in box]
    cropimage = image[max(min(xs),0):max(xs), max(min(ys),0):max(ys)]
    if rect[2]>45:
        cropimage =cropimage.T
    path=os.path.join(dir_path, f"{j}.jpg")
    cropimage=dlib.resize_image(cropimage,33,93)
    cv2.imwrite(path, cropimage)


In [3]:

# load dataset
data_path="./dataset_nomask"
save_path="./dataset_nomasked_eyes"
#process data
detector = dlib.get_frontal_face_detector()
predictor_path = "D:/Data/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
shp_det = dlib.rectangle(0, 0, 149, 149)
os.makedirs(save_path)

for i in range(50):
    dir_path = os.path.join(save_path, f"s{i}")
    os.makedirs(dir_path)
    for j in range(15):
        filepath = f"{data_path}/s{i}/{j}.jpg"
        img = plt.imread(filepath)
        # write the code to detect face in the image (x) using dlib facedetection library
        local_shape = predictor(img, shp_det)
        key_points = local_shape.parts()
        #获得眼部关键点
        temp_points = key_points[17:27]
        temp_points.extend(key_points[36:48])
        temp_points.extend([key_points[27],key_points[28],key_points[0],key_points[16]])
        eye_points = np.ndarray(shape=(0, 2), dtype=np.uint8)
        for p in temp_points:
            eye_points = np.vstack([eye_points, [p.x, p.y]])

        #寻找最小外接矩形、旋转、最后裁切出来
        #找到最小外接矩形
        rect=cv2.minAreaRect(eye_points)
        #box_origin为[(x0,y0),(x1,y1),(x2,y2),(x3,y3)]
        box_origin = cv2.boxPoints(rect)
        #获得旋转矩阵
        M = cv2.getRotationMatrix2D(rect[0],rect[2],1)
        dst = cv2.warpAffine(img,M,(2*img.shape[0],2*img.shape[1]))   
        box = rotatecordiate(rect[2],box_origin,rect[0][0],rect[0][1])
        img=imagecrop(dst,np.int0(box),i,j)
